<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

# LLama 2
In this notebook we are going to learn how to use AWS Bedrock to perform inference using Meta's Llama 2

Let's directly start by creating the bedrock runtime client:


In [1]:
import boto3
import json
bedrock_runtime = boto3.client(aws_access_key_id="",
                               aws_secret_access_key="",
                               region_name="us-east-1",
                               service_name='bedrock-runtime')


### Llama 2 Parameters
Llama 2 accept the following parameters:

- `prompt`: string
  - This represents the initial text or question provided to the system.
- `temperature`: float
  - This controls the randomness in the text generation process. A higher temperature results in more random outputs, while a lower temperature produces more predictable text.
- `top_p`: float
  - This parameter, also known as "nucleus sampling," controls the diversity of the generated text. It sets a threshold to include the most likely next words, cumulatively adding up to the specified probability 'P'. A lower value ignores less probable options. Set to 0 or 1.0 to disable
- `max_gen_len`: int
  - This specifies the maximum number of tokens that the generated text can contain.
   
Thus, the model expects the following jsonified request body:
```
{
    "prompt": "string",
    "temperature": float,
    "top_p": float,
    "max_gen_len": int
}
```

Default Values:


| Category              | Parameter            | JSON object format | Minimum | Maximum | Default |
|-----------------------|----------------------|--------------------|---------|---------|---------|
| Randomness and diversity | Temperature        | `temperature`      | 0       | 1       | 0.5     |
|                       | Top P               | `top_p`            | 0       | 1       | 0.9     |
| Length                | Max generation length| `max_gen_len`      | 1       | 2048    | 512     |



Let's get started!
Note, that the Llama we are using (`meta.llama2-13b-chat-v1`) is optimized for dialog.

In [4]:
prompt = "Explain the normal distribution in the form of a medieval poem please"

In [5]:
body = json.dumps({
    "prompt": prompt,
    "max_gen_len": 256,
    "temperature": 0.9,  # Let's push it to be creative
    "top_p": 0.9,
})


In [7]:
response = bedrock_runtime.invoke_model(body=body, modelId="meta.llama2-13b-chat-v1")
response_body = json.loads(response.get('body').read())
response_body

{'generation': "\n\nIn days of yore, when statistics were scarce\nAnd data was analyzed with little force\nThere lived a knight, a statistician true\nWhose quest was to understand the normal crew\n\nHe studied hard, with pen and ink\nAnd drafted a poem to make it link\nTo the distribution that's so normal and fine\nThe bell-shaped curve, so pure and divine\n\nIn the land of statistics, where data reigns\nThe normal distribution holds the plains\nIt's the most common, the one we adore\nThe distribution that we can't ignore\n\nThe mean, the median, the mode, they all agree\nIn the normal distribution, they're all equal, see\nThe data points cluster, in a curve so neat\nThe normal distribution, a true statistical treat\n\nSo let us praise the normal distribution, so grand\nThe bell-shaped curve, a wondrous land\nWhere data points cluster, in perfect harmony\nThe normal distribution, a true statistical show",
 'prompt_token_count': 14,
 'generation_token_count': 224,
 'stop_reason': 'stop'

Before inspecting the poem, let's quickly look at the response structure:

- `generation`: Generated text
- `prompt_token_count`: Number of tokens in the prompt
- `generation_token_count`: Number of tokens in the generated text
- `stop_reason`: Reason why the response stopped
    - `stop`: Model has finished generating text for the input prompt.
    - `length`: Length of the tokens for the generated text exceeds the value of `max_gen_len`
 
Let's take a look at the poem:

In [9]:
print(response_body["generation"])



In days of yore, when statistics were scarce
And data was analyzed with little force
There lived a knight, a statistician true
Whose quest was to understand the normal crew

He studied hard, with pen and ink
And drafted a poem to make it link
To the distribution that's so normal and fine
The bell-shaped curve, so pure and divine

In the land of statistics, where data reigns
The normal distribution holds the plains
It's the most common, the one we adore
The distribution that we can't ignore

The mean, the median, the mode, they all agree
In the normal distribution, they're all equal, see
The data points cluster, in a curve so neat
The normal distribution, a true statistical treat

So let us praise the normal distribution, so grand
The bell-shaped curve, a wondrous land
Where data points cluster, in perfect harmony
The normal distribution, a true statistical show


What a beautiful poem!

Let's try the same with a very low temperature parameter:

In [10]:
body = json.dumps({
    "prompt": prompt,
    "max_gen_len": 256,
    "temperature": 0,
    "top_p": 0,
})

response = bedrock_runtime.invoke_model(body=body, modelId="meta.llama2-13b-chat-v1")
response_body = json.loads(response.get('body').read())
response_body

{'generation': ".\n\nThe normal distribution, oh so fair,\nA curve that's symmetric, beyond compare.\nIt's bell-shaped, with a peak so high,\nAnd tails that fade, into the sky.\n\nThe mean, a noble knight, doth reign,\nThe standard deviation, a fair maid.\nThe data points, like soldiers in array,\nFollow the knight, in a stately sway.\n\nThe normal distribution, a wondrous sight,\nA pattern of data, that doth ignite.\nIt's the bell-curve of probability,\nA chart of the world, in harmony.",
 'prompt_token_count': 14,
 'generation_token_count': 145,
 'stop_reason': 'stop'}

In [11]:
print(response_body["generation"])

.

The normal distribution, oh so fair,
A curve that's symmetric, beyond compare.
It's bell-shaped, with a peak so high,
And tails that fade, into the sky.

The mean, a noble knight, doth reign,
The standard deviation, a fair maid.
The data points, like soldiers in array,
Follow the knight, in a stately sway.

The normal distribution, a wondrous sight,
A pattern of data, that doth ignite.
It's the bell-curve of probability,
A chart of the world, in harmony.
